In [ ]:
!unzip '/content/ad-dataset.zip'

In [2]:
from tensorflow.keras. layers import Dense, Flatten, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow. keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow. keras.applications.xception import Xception
import numpy as np

ModuleNotFoundError: No module named 'tensorflow'

In [4]:
trainPath = r"/content/Alzheimer_s Dataset/train"
testPath = r"/content/Alzheimer_s Dataset/test"

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
IMG_SIZE = 180
IMAGE_SIZE = [180, 180]
DIM = (IMG_SIZE, IMG_SIZE)
ZOOM = [.99, 1.01]
BRIGHT_RANGE = [0.8, 1.2]
HORZ_FLIP = True
FILL_MODE = "constant"
DATA_FORMAT = "channels_last"
WORK_DIR="/content/Alzheimer_s Dataset/train"
work_dr = IDG(rescale = 1./255, brightness_range=BRIGHT_RANGE, zoom_range=ZOOM, data_format=DATA_FORMAT, fill_mode=FILL_MODE,
horizontal_flip=HORZ_FLIP)
train_data_gen = work_dr.flow_from_directory(directory=WORK_DIR, target_size=DIM, batch_size=6500, shuffle=False)

Found 5121 images belonging to 4 classes.


In [6]:
train_data, train_labels = train_data_gen.next()
#before oversampling
print(train_data.shape, train_labels.shape)

(5121, 180, 180, 3) (5121, 4)


In [7]:
#Performing over-sampling of the data, since the classes are imbalanced
#after oversampling
from imblearn.over_sampling import SMOTE
sm = SMOTE (random_state=42)
train_data, train_labels = sm.fit_resample(train_data.reshape(-1, IMG_SIZE * IMG_SIZE * 3), train_labels)
train_data = train_data.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
print(train_data.shape, train_labels.shape)

(10240, 180, 180, 3) (10240, 4)


In [8]:
#Splitting the data into train, test, and validation sets
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(train_data,	train_labels, test_size = 0.2, random_state=42)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size = 0.2, random_state=42)

In [9]:
IMAGE_SIZE=[180,180]

In [10]:
xcep_model = Xception(input_shape=IMAGE_SIZE+[3], weights='imagenet', include_top=False)

In [11]:
for layer in xcep_model.layers:
    layer.trainable=False

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, GlobalAveragePooling2D
custom_inception_model = Sequential([
        xcep_model,
        Dropout(0.5),
        GlobalAveragePooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout (0.5),
        Dense(64, activation='relu'),
        Dropout (0.5),
        BatchNormalization(),
        Dense(4, activation='softmax')
      ], name = "inception_cnn_model")

In [13]:
import tensorflow
METRICS = [tensorflow.keras.metrics.CategoricalAccuracy(name='acc'),
          tensorflow.keras.metrics.AUC(name='auc')]
custom_inception_model.compile(optimizer='rmsprop',
                                loss=tensorflow.losses.CategoricalCrossentropy(),
                                metrics=METRICS)

In [14]:
history = custom_inception_model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=40)

Epoch 1/40
205/205 [==============================] - 40s 139ms/step - loss: 1.3967 - acc: 0.4201 - auc: 0.6875 - val_loss: 0.9040 - val_acc: 0.6162 - val_auc: 0.8684
Epoch 2/40
205/205 [==============================] - 22s 108ms/step - loss: 0.9389 - acc: 0.5727 - auc: 0.8404 - val_loss: 0.6818 - val_acc: 0.6870 - val_auc: 0.9145
Epoch 3/40
205/205 [==============================] - 23s 111ms/step - loss: 0.7948 - acc: 0.6299 - auc: 0.8830 - val_loss: 0.6505 - val_acc: 0.7004 - val_auc: 0.9208
Epoch 4/40
205/205 [==============================] - 22s 106ms/step - loss: 0.7169 - acc: 0.6750 - auc: 0.9055 - val_loss: 0.6154 - val_acc: 0.7126 - val_auc: 0.9290
Epoch 5/40
205/205 [==============================] - 22s 107ms/step - loss: 0.6764 - acc: 0.7047 - auc: 0.9175 - val_loss: 0.5939 - val_acc: 0.7212 - val_auc: 0.9335
Epoch 6/40
205/205 [==============================] - 22s 107ms/step - loss: 0.6450 - acc: 0.7207 - auc: 0.9255 - val_loss: 0.5605 - val_acc: 0.7383 - val_auc: 0.941

In [15]:
custom_inception_model.save('adp.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [1]:
#import load model class for loading h5 file
from tensorflow.keras.models import load_model
#import image class to process the images
from tensorflow.keras.preprocessing import image
from tensorflow. keras.applications.xception import preprocess_input
import numpy as np
model = load_model('/content/adp.h5')

ModuleNotFoundError: No module named 'tensorflow'

In [17]:
img=image.load_img(r'/content/Alzheimer_s Dataset/train/VeryMildDemented/verymildDem1077.jpg', target_size=(180,180))
x=image.img_to_array(img)
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape
img_data.shape
model.predict(img_data)
output=np.argmax(model.predict(img_data), axis=1)
print(output)

1/1 [==============================] - 0s 32ms/step
[3]
